# Введение в Pandas

К текущему моменту мы уже умеем строить эффективные по быстродействию алгоритмы с участием многомерных массивов, за счет использования NumPy, мы умеем визуализировать данные различными способами. Однако все это время данные, с которыми мы работали, создавались с нуля внутри наших программ. В реальной жизни большая часть задач, решаемых наукой о данных, подразумевает использование данных из вне. Это могут быть данные, полученные в ходе некоторого научного эксперимента, или данные ответов респондентов на вопросы статистического опроса. При этом сами данные могут быть представлены в различных формах (категориальные и числовые данные) или могут даже содержать пропуски. Также сами данные могут храниться в виде простого csv-файла, в формате exel-таблицы или в реляционной базе данных, по типу PostgreSQL.

Возможности работы с основными видами данных, чтения данных из различных источников, а также обработка недостающих данных являются очень важными частями анализа данных. Соответственно, нужен некоторый инструмент, который мог бы удовлетворять нашим требованиям. Первое, что приходит на ум - NumPy. NumPy достаточно эффективно обрабатывает массивы однотипных данных и содержит значительную часть функций, подходящих для статистического анализа. Однако, при пристальном анализе, оказывается, NumPy - не лучший инструмент для работы с сырыми данными. Например, в случае работы с данными, представляющими собой ответы респондентов некоторого статистического опроса, запись в таблице может содержать, как числовые, так и категориальные или даже текстовые данные, например:

|id|age|math grade|physic grade|
|---|---|---|---|
|9f49fef9-8050-4fa5-b910-5f3b9db8ea82|20|8.7|7.9|
|c1058e4c-6163-41c5-a5c0-da3e0b13cbd4|19|5.6|6.8|
|81406e47-ec36-49b5-bbda-656f71747bb9|20|NaN|9.1|

В этом случае мы не сможем эффективно обработать данные разного типа, используя один `np.ndarray`. Более того, даже если мы будем использовать по одному массиву для хранения каждого столбца, мы не сможем осуществлять операции, которые являются необходимыми при анализе денных - объединение таблиц и создание сводных таблиц.

Именно в этот момент на сцену выходит `Pandas`. Pandas не имеет никакой связи с пандами, название фреймворка образовано от словосочетания *Panel data*. Сам Pandas входит в стек Data Science фреймворков, на ряду с NumPy и Matplotlib. Pandas является надстройкой над библиотекой NumPy, реализующей эффективную обработку табличных данных, т.е. разнородных данных, хранение которых организовано в таблицах. Также, по аналогии с NumPy, Pandas обладает эффективной реализацией большого количества функционала по работе с табличными данными. 

![pandas_logo](./images/pandas_logo.png)

## Импорт

По аналогии с NumPy и Matplotlib, Pandas имеет свое стандартное сокращение, которое чаще всего используется в среде аналитиков данных и специалистов в области науки о данных:

In [ ]:
import pandas as pd
import numpy as np

## Основные объекты

Сердцем NumPy является объект `np.ndarray`, который реализует эффективное хранение данных и предоставляет интерфейс для работы с многомерными упорядоченными однотипными данными. Похожие структуры лежат и в основе Pandas, правда, в отличие от NumPy, в Pandas присутствует три основополагающих объекта: `Series`, `DataFrame` и `Index`. Давайте подробнее рассмотрим каждый из этих объектов.

### Index
Начнем наше знакомство с библиотекой Pandas с объекта `pd.Index`. Index является простейшим объектом Pandas и выступает в роли служебной структуры данных, поскольку используется для хранения индексов столбцов и строк в более сложных объектов. По сути своей, Index можно рассматривать как неизменяемый массив из библиотеки NumPy, и этот факт отражает часть интерфейса данного типа данных.

Рассмотрим создание индекса и часть его NumPy-подобного интерфейса:

In [ ]:
index_int = pd.Index(list(range(10)))
index_str = pd.Index(list("abc"))
index_obj = pd.Index([(1, ), (2, ), {3, 4}])

print(
    f"index int:\n{index_int};",
    f"index str:\n{index_str};",
    f"index obj:\n{index_obj};",
    sep="\n",
    end="\n\n"
)

print(
    f"index size: {index_int.size};",
    f"index shape: {index_int.shape};",
    f"index ndim: {index_int.ndim};",
    f"index dtype: {index_int.dtype};",
    sep="\n",
)

Также, по аналогии с массивами NumPy, индексы Pandas для получения данных могут быть проиндексированы целыми числами, срезами, массивами индексов, а также булевыми масками. Однако, стоит повторно отметить тот факт, что индексы в Pandas являются неизменяемыми типами данных, т.е. не поддерживает операции изменения значений элементов. Проиллюстрируем эти утверждения примерами:

In [ ]:
index = pd.Index(list(range(10)))

print(f"index:\n{index}", end="\n\n")

print(
    f"int index: {index[0]};",
    f"slice index: {index[::3]};",
    f"array index: {index[[1, 5, 6]]};",
    f"boolean mask index: {index[index < 5]};",
    sep="\n",
    end="\n\n",
)

try:
    index[0] = 42

except TypeError as exception:
    print(exception)

Неизменяемость индекса необходима для совместного безопасного использования одного объекта индекса несколькими объектами типов `pd.DataFrame` или `pd.Series`. Неизменяемость гарантирует, что ни один из объектов, использующих индекс, не сможет внести в него изменения, ломающие логику другого объекта, использующего тот же индекс.

Помимо интерфейса NumPy-массива индексы Pandas также обладают интерфейсом множеств, и могут быть определены как упорядоченное мультимножество. Термин "мультимножество" подразумевает тот факт, что любой индекс может содержать несколько элементов с одним и тем же значением. Наличие интерфейса множеств необходимо для реализации выполнения различных операций между более сложными объектами Pandas, что будет рассмотрено ниже. Рассмотрим несколько примеров работы с индексами Pandas, как с мультимножествами.

In [ ]:
index1 = pd.Index([1, 2, 2, 4, 6])
index2 = pd.Index(list(range(index1.size)))

print(
    f"index1:\n{index1}",
    f"index2:\n{index2}",
    sep="\n",
    end="\n\n",
)

print(
    f"intersection: {index1.intersection(index2)};",
    f"union: {index1.union(index2)};",
    f"difference I1 - I2: {index1.difference(index2)}",
    f"difference I2 - I1: {index2.difference(index1)}",
    f"symmetrical difference: {index1.symmetric_difference(index2)};",
    sep="\n",
)

### Series

Следующий объект в иерархии объектов Pandas - это объект `pd.Series`, - одномерный массив индексированных данных. Простейший способ создания этого объекта - это создание из списка.

In [ ]:
series = pd.Series(list(range(10)))
print(f"series:\n{series}")

Как видно из данного листинга кода, объект Series является адаптером для самой последовательности значений, а также для индексов, которые могут быть переданы явно, или, как в данном случае, могу быть сгенерированы автоматически. Мы можем получить доступ как к значениям индекса, так и к значениям самих данных, используя специальные атрибуты объекта Series. 

In [ ]:
print(
    f"series index:\n{series.index}",
    f"series values:\n{series.values}",
    sep="\n\n",
)

Как мы видим, индекс хранится в уже знакомом нам объекте `pd.Index`, значения же хранятся в также знакомом нам `np.ndarray`. Из этих фактов следует, что `Series`является адаптером не просто для индекса и последовательности значений, а адаптером для однотипного индекса и для последовательности значений одного типа.

Мы можем думать об объекте Series, как об обобщенном одномерном массиве NumPy, поскольку Series является упорядоченной, изменяемой, явно индексируемой последовательностью данных одного типа. За счет задания явного индекса и возможности использования не только числовых значений для составления индекса, объект Series значительно расширяет возможности одномерных массивов NumPy.

In [ ]:
series = pd.Series(
    data=[1, 2, 3],
    index=["first", "second", "third"],
)
print(f"series:\n{series}", end="\n\n")
print(
    f"int indexing: {series[0]};",
    f"custom index indexing: {series['first']};",
    sep="\n",
)

Обратим внимание на тот факт, что и индексы и сами данные должны быть одного типа. В случае несоответствия типов Pandas, по аналогии с NumPy, будет осуществлять повышающее приведение типов. Если переданные объекты не могут быть приведены к одному общему типу, Pandas создаст коллекцию с `dtype=object`, т.е. с самым общим типом объектов Python.

Также индексы могут содержать не обязательно логически упорядоченные и полные данные для индексирования Series:

In [ ]:
series = pd.Series(
    data=[1, 2, 3],
    index=[1, 5, 3],
)
print(f"series:\n{series}")

При задании индексов и данных в виде упорядоченных последовательностей, Pandas устанавливает соответствия данных индексам по их позициям, и сохраняет их в Series в переданном порядке следования элементов.

Так же мы можем думать о `Series`, как о специализированной версии словаря со строго типизированными значениями и ключами. Для того, чтобы эта аналогия была более явной, сконструируем объект Series из словаря Python.

In [ ]:
population_dict = {
    "Moscow": 13149803,
    "Saint Petersburg": 5600044,
    "Novosibirsk": 1635338,
    "Ekaterinburg": 1539371,
    "Kazan": 1495066,
}

populations = pd.Series(population_dict)
print(populations)

При подобном способе создания объекта Series данные будут следовать в порядке следования ключей словаря в сохраненной в памяти хэш-таблице.

Поскольку объект Series гораздо сложнее описанных выше объектов Index, рассмотрим способы создания Series подробнее: 

In [ ]:
# из массива с автоматической генерацией индекса на стороне Pandas
print(f"auto generated index:\n{pd.Series(list(range(5)))}", end="\n\n")

# с указанием fill_value и последовательности индексов
print(f"with fill_value:\n{pd.Series(5, index=list('abc'))}", end="\n\n")

# из словаря
print(f"from dict:\n{pd.Series({'a': 1, 'b': 2, 'c': 3})}", end="\n\n")

# из словаря с указанием индексов
print(
    "from dict with filter:",
    f"{pd.Series({'a': 1, 'b': 2, 'c': 3}, index=['a', 'c'])}",
    sep="\n"
)

### DataFrame

Последний объект в иерархии объектов Pandas - это объект `DataFrame`. Объект DataFrame является аналогом двумерного массива NumPy с возможностью явного указания индексов строк и столбцов. Более точно, DataFrame можно определить так: DataFrame - это упорядоченная последовательность выровненных объектов Series. Под выравниванием здесь понимается тот факт, что все элементы DataFrame используют один и тот же объект Index для индексации. Рассмотрим пример: создадим два объекта типа `pd.Series`, разделяющих общие индексы, и создадим с их помощью объект DataFrame.

In [ ]:
populations = pd.Series(
    {
        "Moscow": 13149803,
        "Saint Petersburg": 5600044,
        "Novosibirsk": 1635338,
        "Ekaterinburg": 1539371,
        "Kazan": 1495066,
    }
)
areas = pd.Series(
    {
        "Saint Petersburg": 1439,
        "Kazan": 515.8,
        "Novosibirsk": 502.7,
        "Ekaterinburg": 1112,
        "Moscow": 2511,
    }
)

city_stats = pd.DataFrame(
    {
        "population": populations,
        "area": areas,
    }
)
city_stats

DataFrame, по аналогии с Series, имеет атрибуты для прямого доступа к содержимому таблицы, именам колонок и именам строк: 

In [ ]:
print(
    f"columns:\n{city_stats.columns}",
    f"rows:\n{city_stats.index}",
    f"data:\n{city_stats.values}",
    sep="\n\n"
)

Атрибуты `index` и `columns` возвращают объект типа Index, атрибут `values` - np.ndarray.

Объект DataFrame может быть интерпретирован, как словарь с типизированными ключами, значениями которого являются объекты Series. Это может быть продемонстрировано на следующем примере:

In [ ]:
print(f"{city_stats['area']}")

Обратите внимание, что в отличие от NumPy, для объектов DataFrame индексация по умолчанию осуществляет по столбцам, а не по строкам.

Рассмотрим самые популярные способы создания DataFrame, так же, как мы это делали с Series.

In [ ]:
# из объекта Series
print(
    f"from Series:\n{pd.DataFrame(populations, columns=['population'])}",
    end="\n\n",
)

# из списка словарей Python
data = [{1: i, 2: i * 2} for i in range(1, 4)]
print(
    f"from dicts:\n{pd.DataFrame(data, index=range(1, 4))}",
    end="\n\n",
)

# из словаря объектов Series
data = {
    "population": populations,
    "area": areas,
}
print(
    f"from dict of Series:\n{pd.DataFrame(data)}",
    end="\n\n",
)

# из двумерного массива NumPy
data_frame = pd.DataFrame(
    np.random.randint(0, 255, size=(2, 2)),
    index=["row1", "row2"],
    columns=["col1", "col2"],
)
print(f"from 2D-array:\n{data_frame}")

### Итог

|Объект|Изменяемость|Типизированность|Похож на|
|---|---|---|---|
|Index|Неизменяем|Типизирован|Одномерный массив NumPy или упорядоченное типизированное мультимножество|
|Series|Изменяем|Типизированные ключи и значения|Упорядоченный словарь со строго типизированными ключами и значениями или одномерный массив NumPy с обобщенной типизацией|
|DataFrame|Изменяем|Типизированные ключи и значения|Упорядоченный словарь элементов Series, со строго типизированными ключами|

## Выборка данных из объектов Pandas

За счет специфики объекты Pandas обладают специальным функционалом для осуществления выборки данных. В предыдущем разделе мы уже обсудили выборку данных из объекта `pd.Index`, поскольку этот объект наиболее приближен к знакомым нам массивам NumPy. В этом разделе подробнее рассмотрим способы выборки данных и индексации объектов Series и DataFrame.

### Series

Как было сказано ранее, объект Series во многом похож на словарь. Эта аналогия отчасти поможет нам понять его поведение и паттерны индексации. Подобно словарю объект Series реализует специальные методы `__getitem__` и `__contains__`:

In [ ]:
series = pd.Series(
    [1 - 2 ** (i * -1) for i in range(4)],
    index=list("abcd"),
)

print(f"series:\n{series}", end="\n\n")

print(
    f"__contains__: {'b' in series};",
    f"__geitem__: {series['c']};",
    sep="\n",
)

Так же, как и в случае словаря, мы можем итерироваться по Series, перебирая только ключи, только значения или пары ключ-значение:

In [ ]:
string_data = ", ".join(str(elem) for elem in series)
string_keys = ", ".join(str(key) for key in series.keys())
string_pairs = ", ".join(str(pair) for pair in series.items())

print(
    f"default iterations:\n{string_data}",
    f"keys iterations:\n{string_keys}",
    f"pairs iterations:\n{string_pairs}",
    sep="\n\n",
)

Здесь стоит сделать несколько замечаний. Первое: в отличие от словаря в Python для объекта Series итерирование по умолчанию осуществляется по значениям, а не по ключам. Второе замечание, метод `keys()` объекта Series возвращает  объект Index. Последнее замечание: метод `items()`, подобно аналогичному методу словарей, возвращает генератор пар и эквивалентен следующему вызову объекта `zip()`:  

In [ ]:
for pair in zip(series.index, series.values):
    print(pair)

Помимо выборки данных, мы можем добавлять новые данные в объект Series, используя синтаксис словарей:

In [ ]:
series = pd.Series(
    data=list(range(4)),
    index=list("abcd"),
)
print(f"series\n{series}", end="\n\n")

series["e"] = 5
print(f"series:\n{series}", end="\n\n")

В этом случае Pandas будет дописывать данные в конец последовательности.

Теперь рассмотрим индексацию объекта Series в стиле массивов NumPy. В примерах выше мы видели, что объект Series может быть индексирован с помощью целочисленных индексов, или с помощью явных значений индекса, если он был определен. Рассмотрим другие доступные способы индексации:

In [ ]:
series = series = pd.Series(
    data=list(range(5)),
    index=list("abcde"),
)

In [ ]:
print(
    # срез с помощью явного индекса
    f"explicit index:\n{series['b':'d']}",
    # срез с помощью неявного индекса
    f"implicit index:\n{series[1:3]}",
    # индексация с помощью массива индексов
    f"array of indices:\n{series[['a', 'c', 'e']]}",
    # индексация с помощью булевой маски
    f"boolean mask index:\n{series[series < 3]}",
    sep="\n\n",
)

Здесь стоит обратить внимание на разницу между срезом с помощью явного индекса и срезом с помощью неявного индекса. Срез, получаемый посредством явного индекса, включает правую границу срезу. В примере выше элемент под индексом `'d'` был включен в результата. Срез посредством неявного индекса работает аналогично срезам списков в Python, т.е. правая граница среза в результат не включается.

#### Индексаторы

Возможность использования явного и неявного индекса для выборки данных может нередко может приводить к путанице и ошибкам. Рассмотрим следующий пример: 

In [ ]:
series = pd.Series(data=[1, 2, 3], index=[1, 3, 5])
print(f"series:\n{series}")

In [ ]:
# Pandas использует явный индекс
print(series[1])

# Pandas использует неявный индекс
print(series[1:3])

Как мы видим в примере выше, при использовании целых чисел в качестве явного индекса возникает состояние неопределенности. В каких-то случаях Pandas использует явный индекс для доступа к данным, а в каких-то случаях воспринимает переданные индексы, как неявные индексы, и вместо элементов под индексами 1 и 3 возвращает не то, что мы ожидали. Но как быть? Неужели при использовании целочисленных индексов у нас нет возможности выборки данных с их помощью? Такая возможность есть, и чтобы ею воспользоваться нам придется познакомиться с индексаторами.

Не стоит пугаться, индексаторы не представляют из себя ничего сложного, по факту, это всего лишь атрибуты объектов Pandas, которые позволяют явно сообщить интерпретатору, каким именно индексом вы намерены пользоваться: явным или неявным. Начнем знакомство с индексаторами с атрибута `loc`. Атрибут loc позволяет сообщить интерпретатору, что вы намерены использовать явный индекс. Перепишем предыдущий листинг кода с использованием атрибута `loc`.  

In [ ]:
series = pd.Series(data=[1, 2, 3], index=[1, 3, 5])
print(f"series:\n{series}")

In [ ]:
print(series.loc[1])
print(series.loc[1:3])

Как видим теперь и в первом, и во втором вызове данные были выбраны в соответствии с явно указанным индексом. Атрибут `iloc`, напротив, позволяет сообщить интерпретатору, что вы намерены использовать неявный индекс для доступа к данным:

In [ ]:
print(series.iloc[1])
print(series.iloc[1:3])

Напомним, что одно из положений Дзена Python гласит: "Лучше явное, чем неявное". Использование индексаторов позволяет не только избежать ошибок и путаницы, но и сделать ваш код более читабельным и понятным. Поэтому старайтесь обращаться к элементам объектов Pandas с помощью индексаторов и избегайте обращения напрямую через квадратные скобки (тем более в версии библиотеки v2 данное поведение считается устаревшим, и разработчики намерены удалить эту возможность в будущих версиях фреймворка).

### DataFrame

Способы индексации DataFrame во многом похожи на способы индексации Series. Однако, очевидно, DataFrame обладает своей спецификой при выборке данных. Выше мы уже видели использование синтаксиса словарей при обращении к элементам DataFrame. Помимо этого мы можем обращаться к элемента DataFrame, как к атрибутам объекта: 

In [ ]:
populations = pd.Series(
    {
        "Moscow": 13149803,
        "Saint Petersburg": 5600044,
        "Novosibirsk": 1635338,
        "Ekaterinburg": 1539371,
        "Kazan": 1495066,
    }
)
areas = pd.Series(
    {
        "Saint Petersburg": 1439,
        "Kazan": 515.8,
        "Novosibirsk": 502.7,
        "Ekaterinburg": 1112,
        "Moscow": 2511,
    }
)

city_stats = pd.DataFrame(
    {
        "population": populations,
        "area": areas,
    }
)
city_stats

In [ ]:
print(
    f"dict style:\n{city_stats['population']}",
    f"attribute style:\n{city_stats.population}",
    sep="\n\n",
    end="\n\n",
)

print(city_stats["population"] is city_stats.population)

Однако, несмотря на такую возможность, старайтесь избегать обращений к данным в стиле атрибутов объекта, поскольку, в случае конфликта имен между именем столбца и именем атрибута объекта DataFrame вы рискуете получить ошибку. Особенно избегайте использования данного стиля работы с данными DataFrame при создании новых столбцов, чтобы не переопределить существующий атрибут объекта DataFrame. Старайтесь придерживаться синтаксиса в стиле словарей:

In [ ]:
city_stats["population density"] = (
    city_stats["population"] / city_stats["area"]
)
city_stats

Зная о пользе индексаторов при работе с объектами Pandas, рассмотрим индексацию объекта DataFrame с их использованием. Итак, с помощью атрибута `iloc`, мы можем осуществлять индексацию с использованием неявных целочисленных индексов. В этом случае выборка данных будет похожа на выборку данных из двумерного массива, за исключением сохранения имен строк и столбцов. Рассмотрим эти утверждения на примере:

In [ ]:
slice_row, slice_col = slice(0, 3), slice(0, 2)

array_data = city_stats.values
print(f"array data:\n{array_data}", end="\n\n")

print(f"array slice:\n{array_data[slice_row, slice_col]}", end="\n\n")
city_stats.iloc[slice_row, slice_col]

Того же самого мы могли бы добиться, используя явный индекс вместе с атрибутом `loc`.

In [ ]:
city_stats.loc[:"Moscow", :"area"]

Также оба вида индексаторов поддерживают индексацию с помощью булевых масок и массивов значений. Рассмотрим примеры:

In [ ]:
city_stats.loc[city_stats["area"] > 1e3, ["population", "area"]]

In [ ]:
city_stats.iloc[city_stats.values[:, 1] > 1e3, [0, 1]]

В завершении отметим дополнительный синтаксис индексации объекта DataFrame. Во-первых, простые срезы без использования индексаторов относятся к строкам, а не к столбцам: 

In [ ]:
city_stats[1:3]

Во-вторых использование булевых масок также относится к строкам, а не столбцам:

In [ ]:
city_stats[city_stats["area"] < 1e3]

Данный синтаксис плохо стыкуется с изученным поведением объектов Pandas и тем фактом, что первым измерением у DataFrame являются столбцы, а не строки. Однако, поскольку Pandas создавался для удобства манипуляции табличными данными, люди, знакомые с реляционными СУБД и языком запросов SQL, могут найти здесь параллели с выборкой данных из таблиц с использованием условия `WHERE` или ограничения вывода с помощью `LIMIT` и `OFFSET`. 